In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import cvxpy as cp

import torch
from torch.autograd import Variable

from free_flyer.free_flyer import FreeFlyer
import pickle, os

In [2]:
#load train/test data
prob = FreeFlyer() #use default config, pass different config file oth.
config_fn = './free_flyer/config/default.p'

config_file = open(config_fn,'rb')
dataset_name, _, _ = pickle.load(config_file); config_file.close()

relative_path = os.getcwd()
dataset_fn = relative_path + '/free_flyer/data/' + dataset_name

train_file = open(dataset_fn+'/train.p','rb')
# p_train, x_train, u_train, y_train, c_train, times_train = pickle.load(train_file)
train_data = pickle.load(train_file)
train_file.close()

test_file = open(dataset_fn+'/test.p','rb')
# p_test, x_test, u_test, y_test, c_test, times_test = pickle.load(test_file)
test_data = pickle.load(test_file)
test_file.close()

# test task-specific MLOPT

In [3]:
from solvers.mlopt_ff import MLOPT_FF

In [4]:
system = 'free_flyer'
prob_features = ['x0', 'obstacles', 'obstacles_map']

mlopt_ts_obj = MLOPT_FF(system, prob, prob_features)

n_features = 36 + prob.n_obs
mlopt_ts_obj.construct_strategies(n_features, train_data)
print(mlopt_ts_obj.n_strategies)

379


In [5]:
mlopt_ts_obj.setup_network()

# cnn_in = Variable(torch.from_numpy(mlopt_ts_obj.cnn_features[:2])).float()
# ff_in = Variable(torch.from_numpy(mlopt_ts_obj.features[:2])).float()

# mlopt_ts_obj.model(cnn_in, ff_in).detach().cpu().numpy()
print(type(mlopt_ts_obj.model))

<class 'pytorch.models.CNNet'>


In [6]:
mlopt_ts_obj.training_params['TRAINING_ITERATIONS'] = 500
mlopt_ts_obj.train(verbose=True)

loss:   3.172037363052368,   acc:  0.15625
Done with epoch 0 in 5.178166627883911s
loss:   2.665426015853882,   acc:  0.28125
Done with epoch 1 in 4.988928318023682s
loss:   2.4943110942840576,   acc:  0.28125
loss:   2.5960640907287598,   acc:  0.25
Done with epoch 2 in 5.0579986572265625s
loss:   2.933814287185669,   acc:  0.28125
Done with epoch 3 in 5.01059627532959s
loss:   3.0009925365448,   acc:  0.09375
loss:   3.0832772254943848,   acc:  0.375
Done with epoch 4 in 5.040608644485474s
loss:   3.2641663551330566,   acc:  0.125
Done with epoch 5 in 5.076635360717773s
loss:   3.254560708999634,   acc:  0.125
Done with epoch 6 in 5.005603075027466s
loss:   2.4300215244293213,   acc:  0.25
loss:   2.8489205837249756,   acc:  0.40625
Done with epoch 7 in 5.015244245529175s
loss:   3.4127659797668457,   acc:  0.21875
Done with epoch 8 in 4.964863300323486s
loss:   2.597259998321533,   acc:  0.21875
loss:   2.5296313762664795,   acc:  0.3125
Done with epoch 9 in 5.040590524673462s
loss:

KeyboardInterrupt: 

In [ ]:
prob_params = {}
idx = np.random.randint(test_data[1].shape[0])

p_test = test_data[0]
for k in p_test.keys():
    prob_params[k] = p_test[k][idx]
prob_success, cost, total_time, n_evals = mlopt_ts_obj.forward(prob_params)

if prob_success:
    print(cost)
else:
    print('Failed')

In [ ]:
prob_success, true_cost, solve_time, optvals = mlopt_ts_obj.problem.solve_micp(prob_params)

true_cost

In [ ]:
x0 = train_data[0]['x0'][idx]
n_obs = mlopt_ts_obj.problem.n_obs
posmin, posmax = mlopt_ts_obj.problem.posmin, mlopt_ts_obj.problem.posmax

obstacles = []
for ii_obs in range(n_obs):
    obs = train_data[0]['obstacles'][idx][:,ii_obs]
    obstacles.append(obs)

X = optvals[0]

plt.axes()
for obstacle in obstacles:
    rectangle = plt.Rectangle((obstacle[0], obstacle[2]), \
                              obstacle[1]-obstacle[0], obstacle[3]-obstacle[2], \
                             fc='red', ec='blue')
    plt.gca().add_patch(rectangle)
    plt.axis('scaled')

for ii in range(mlopt_ts_obj.problem.N):
    circle = plt.Circle((X[0,ii],X[1,ii]), 0.04, fc='blue',ec="green")
    plt.gca().add_patch(circle)

ax = plt.gca()
ax.margins(0)
ax.set(xlim=(posmin[0],posmax[0]), ylim=(posmin[1],posmax[1]))
plt.show()